density1_estimation_all_region_XXslices_average_reconstcords_voxvol
- density1: official pipeline step 1
- estimation: we only have access to a subset of the cells
- all_region: whole mouse brain (for now: except 3 problematic regions)
- XX slices: MERFISH slices only 53
- average: we don't split sides, but we combine slices for average if a region is crossed by multiple slices (not here, in density2)
- reconstcords: we use the reconstructed coordinates from Allen (and not CCF or RAW coords)
- voxvol: we estimate the area/volume using voxelised crossection between slice and CCFv3 (done in density0 as cells df)

In [1]:
import os, sys, shutil
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import requests
from scipy.spatial import Delaunay
import nrrd
from voxcell import RegionMap
#import pickle

import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import calculate_density_mm3

## Helper functions

## Load data

In [2]:
%%time 

version = '20231215'
download_base = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/'

use_local_cache = False
manifest_path = 'releases/%s/manifest.json' % version

if not use_local_cache :
    url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/' + manifest_path
    manifest = json.loads(requests.get(url).text)
else :
    file = os.path.join(download_base,manifest_path)
    with open(file,'rb') as f:
        manifest = json.load(f)
        
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['MERFISH-C57BL6J-638850-CCF']['directories']['metadata']['relative_path'], 
                              'views')

file = os.path.join( view_directory, 'cell_metadata_with_parcellation_annotation.csv')

#Load all MERFISH spatial data into 1 dataframe
cell_joined = pd.read_csv(file)
cell_joined.set_index('cell_label',inplace=True)
print(f"All MERFISH cells: {cell_joined.shape[0]}")

#Load voxelised region volume for each cell (note that this has less cells) 
view_directory = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/nmi_scores/'
file = os.path.join( view_directory, 'cells_in_respective_volumes.csv')
cells = pd.read_csv(file)
cells.set_index('cell_label',inplace=True)

# Combine the two dfs by removing 15 problematic cells and adding relevant columns
# Identify common rows based on a unique identifier column
common_rows = cells.index.intersection(cell_joined.index)
# Filter cells_joined to retain only common rows
cell_joined_filtered = cell_joined.loc[common_rows]
# Specify the columns to add to cells_joined
columns_to_add = ['template_nr', 'parcellation_substructure', 'parcellation_term_name', 'region_id', 'voxel_vol', ]
# Merge cells_joined with the selected columns from cells based on the index
cell_joined = cell_joined_filtered.merge(cells[columns_to_add], left_index=True, right_index=True, how='left')
print(f"Filtered MERFISH cells: {cell_joined.shape[0]}")

#We can make the main df lighter for the loop
columns_to_drop = ['cluster_alias', 'average_correlation_score', 'feature_matrix_label', 
                   'donor_label', 'donor_genotype', 'donor_sex', 'neurotransmitter_color', 
                   'class_color', 'subclass_color', 'supertype_color', 'cluster_color', 
                   'parcellation_organ', 'parcellation_category', 'parcellation_division',
                   'parcellation_structure', 'parcellation_organ_color', 
                   'parcellation_category_color', 'parcellation_division_color', 
                   'parcellation_structure_color', 'parcellation_substructure_color', 
                   'class', 'subclass', 'supertype', 'x_ccf', 'y_ccf', 'z_ccf', ]
cell_joined.drop(columns=columns_to_drop, inplace=True)
print(f"Columns dropped to {cell_joined.shape[1]}")

#Load all parcellation data with extended info instead
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)

#This describes all indexes in the slices, but also on every level: organ category division structure substructure
parcellation_indexes = list(np.unique(cell_joined['parcellation_index']))
description_of_all_indexes = parcellation_annotation[parcellation_annotation['parcellation_index'].isin(parcellation_indexes)]

#Create smaller dataframes
substructure_info = description_of_all_indexes[description_of_all_indexes['parcellation_term_set_name'] == 'substructure'] 

hierarchy_json = '/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(hierarchy_json)

All MERFISH cells: 3739961
Filtered MERFISH cells: 3791571
Columns dropped to 16
CPU times: user 52.1 s, sys: 4.31 s, total: 56.4 s
Wall time: 57.6 s


In [3]:
# Check if the DataFrame contains any NaN values
has_nan = cell_joined.isna().any().any()
#has_nan = cell_joined['voxel_vol'].isna().any()

if has_nan:
    print("The DataFrame contains NaN values.")
    
    # Get the columns with NaN values
    columns_with_nan = cell_joined.columns[cell_joined.isna().any()].tolist()
    print("Columns with NaN values:", columns_with_nan)
    
else:
    print("The DataFrame does not contain any NaN values.")


The DataFrame contains NaN values.
Columns with NaN values: ['neurotransmitter']


parcellation_substructure _x and _y come from the 2 dfs, they are the same

In [4]:
output_dir_csv = f"{download_base}results/density_calculations/csv"
output_dir_log = f"{download_base}results/density_calculations/log"
output_dir_nrrd = f"{download_base}results/density_calculations/nrrd"

# Confirm the directory path before deletion
if "csv" in output_dir_csv and os.path.exists(output_dir_csv):
    shutil.rmtree(output_dir_csv)
    shutil.rmtree(output_dir_log)
    print(f"Deleted directories: {output_dir_csv} {output_dir_log}")
else:
    print("Directory path is not correct or does not exist.")

os.makedirs(output_dir_csv, exist_ok=True)
os.makedirs(output_dir_log, exist_ok=True)
os.makedirs(output_dir_nrrd, exist_ok=True)

print(f"Directory {output_dir_csv[99:]} and {output_dir_log[99:]} and {output_dir_nrrd[99:]} has been recreated or exist.")

Deleted directories: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/csv /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/log
Directory csv and log and nrrd has been recreated or exist.


## Loop

- The next loop will open a file for logging. 
- It will loop through all 677 region and filter the cell dataframe (all 53 slices) based on these regional info. It will loop and print all extra substructures (leaf regions in the Cerebellum and 1 in the OLF) we added to the metadata. 
- It will create a csv files with densities of each cell type in a region. Filename: region name + density + two_sides (meaning if the code does not need to take into consideration whether two cells exist in two patches (i.e. the region exist on both hemispheres) or one. 

In [5]:
%%time
all_section_count = len(np.unique(cell_joined['brain_section_label']))
# edge_length_mm = 10 * 0.001 # edge length of a 10 um3 voxel in mm
# volume_single_cube_mm3 = edge_length_mm ** 3 #vol of a voxel in mm3
volume_single_cube_mm3 = 1e-06

# Specify the full or relative path to the log file
root_folder = f"{download_base}results/density_calculations/"
log_file_path = f"{root_folder}/log/print_log_density1_part.txt"


CPU times: user 2.9 s, sys: 7.89 ms, total: 2.91 s
Wall time: 2.9 s


In [6]:
%%time

# Open the file in append mode
with open(log_file_path, "w") as log_file:
    for selected_region in substructure_info['parcellation_term_acronym'][0:]: #Selecting cell type
        
        #print(selected_region) 
        #The base df is in the next line: all cells belonging to one region across many sections
        cells_in_region = cell_joined[cell_joined['parcellation_substructure_x'] == selected_region]
        nr_of_cells = cell_joined[cell_joined['parcellation_substructure_x'] == selected_region].shape[0]
        sections = len(np.unique(cells_in_region['brain_section_label']))       
        ctypes_in_region = len(np.unique(cells_in_region['cluster']))
        #region_id = int(substructure_info[substructure_info['parcellation_term_acronym'] == 'VISpl6b']['parcellation_label'].values[0].split('-')[-1])
        #region_id = set(cells_in_region['region_id'].values)
        log_file.flush()  # Flush the buffer to ensure immediate write
        print("", file=log_file)
        print(f'There are {nr_of_cells} cells found in the {selected_region} which belong into {ctypes_in_region} cell types', file=log_file)
        print(f'There are {sections} sections which intersect the {selected_region} out of {all_section_count}.', file=log_file)
        #coronal_slice_positions = np.unique(cells_in_region['z_reconstructed'])
        cluster_as_filename = selected_region.translate(str.maketrans({"/":  r"", "-":  r"", " ":  r"", ",": r""}))
        result_df = calculate_density_mm3(cells_in_region, volume_single_cube_mm3, selected_region)
        if not result_df.empty:
            result_df.to_csv(f"{root_folder}csv/{cluster_as_filename}_density_two_sides.csv", index=True)
        else:
            print(f"{selected_region} has an empty df {sections} sections and {nr_of_cells} cells found..")

MOBglomerularlayer has an empty df 0 sections and 0 cells found..
CENT2granularlayer has an empty df 0 sections and 0 cells found..
CENT2purkinjelayer has an empty df 0 sections and 0 cells found..
CENT2molecularlayer has an empty df 0 sections and 0 cells found..
CENT3granularlayer has an empty df 0 sections and 0 cells found..
CENT3purkinjelayer has an empty df 0 sections and 0 cells found..
CENT3molecularlayer has an empty df 0 sections and 0 cells found..
ANcr1granularlayer has an empty df 0 sections and 0 cells found..
ANcr1purkinjelayer has an empty df 0 sections and 0 cells found..
ANcr1molecularlayer has an empty df 0 sections and 0 cells found..
ANcr2granularlayer has an empty df 0 sections and 0 cells found..
ANcr2purkinjelayer has an empty df 0 sections and 0 cells found..
ANcr2molecularlayer has an empty df 0 sections and 0 cells found..
LINGgranularlayer has an empty df 0 sections and 0 cells found..
LINGpurkinjelayer has an empty df 0 sections and 0 cells found..
LINGmole

In [7]:
import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import get_all_filenames, get_csv_filenames, extract_prefix_from_filenames


In [8]:
# Get all regional density data
folder_path = f"{root_folder}csv/"
filenames = get_all_filenames(folder_path)
csv_filenames = get_csv_filenames(folder_path)
prefixes = extract_prefix_from_filenames(csv_filenames)
unique_prefixes = sorted(list(set(prefixes)))

In [9]:
#Max leaf region length: 677
len(csv_filenames)

670

In [10]:
def read_and_test_csv_files(filenames, unique_prefixes, folder_path):
    '''Test all files created above, whether they are empty and the columns are in order.'''
    for prefix in unique_prefixes:
        matching_files = [filename for filename in filenames if filename.startswith(prefix)]
        if matching_files:
            for filename in matching_files:
                try:
                    # Read the CSV file
                    df = pd.read_csv(os.path.join(folder_path, filename), index_col='cluster')
                except StopIteration:
                    print(f"StopIteration encountered when reading {filename}. The file might be corrupted or incorrectly formatted.")
                    continue  # Skip to the next file
                except Exception as e:
                    print(f"Error reading {filename}: {e}")
                    continue  # Skip further checks if reading failed
                
                if df.empty:
                    print(f"The DataFrame from {filename} is empty")
                    continue  # Skip further checks if the DataFrame is empty

                # Check if 'cluster' is the index and 'density_mm3' is a column
                if df.index.name != 'cluster' or 'density_mm3' not in df.columns:
                    print(f"The DataFrame from {filename} has incorrect index or missing 'density_mm3' column")
                    continue  # Skip further checks if the index or column is incorrect
                
                # Convert 'density_mm3' to numeric if it's not already
                try:
                    df['density_mm3'] = pd.to_numeric(df['density_mm3'], errors='raise')
                except ValueError:
                    print(f"Cannot convert 'density_mm3' to numeric in {filename}. Data might be corrupted or invalid.")
                    continue  # Skip further checks if conversion fails

                # Check data types of 'cluster' and 'density_mm3' columns
                if df.index.dtype != object or not pd.api.types.is_numeric_dtype(df['density_mm3']):
                    print(f"The DataFrame from {filename} has incorrect data types")
                    continue  # Skip further checks if data types are incorrect

                # If all checks pass, store the DataFrame
                # result_dataframes[filename] = df

    print("If there are no print messages before this, all DataFrames / csv files contain cells and densities.")

read_and_test_csv_files(csv_filenames, unique_prefixes, folder_path)

If there are no print messages before this, all DataFrames / csv files contain cells and densities.


In [11]:
print("done")

done


# Test next step (density2 step): 
density_comp_nrrd_jobarray.sh + density_comp_nrrd_on_1node.py

In [16]:
import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import get_all_filenames, get_csv_filenames, extract_prefix_from_filenames, get_nrrd_files, nrrd_from_df
from helper_functions import read_and_concat_csv_files_new, combine_rows_and_calculate_average, create_combined_dataframe

In [17]:
#Load all csv files with densities
root_folder = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'
folder_path = f'{root_folder}csv/'
filenames = get_all_filenames(folder_path)
csv_filenames = get_csv_filenames(folder_path)
prefixes = extract_prefix_from_filenames(csv_filenames)
unique_prefixes = sorted(list(set(prefixes)))


In [ ]:
%%time
#Create a dict of df, each containing a cell type's occurence in all regions and its densities in all regions
result_dataframes = read_and_concat_csv_files_new(csv_filenames, unique_prefixes, folder_path)
combined_result_dataframes = combine_rows_and_calculate_average(result_dataframes)
shuffled_combined_dataframes = create_combined_dataframe(combined_result_dataframes)


In [ ]:
import re 

# Get the set of all clusters present in the result_dataframes
all_clusters = set()
for df in result_dataframes.values():
    all_clusters.update(df.index)

cluster_ids = []
for cluster in all_clusters: 
    match = re.search(r'\d+', cluster)
    if match:
        # Extract the first 4 numbers
        cluster = match.group()[:4]
    else:
        # If no numeric portion is found, keep the original string
        cluster = cluster
        print(f"No numeric portion is found, keeping original name for: {cluster}", flush=True)
    
    cluster_ids.append(cluster)
     
cluster_ids = sorted(cluster_ids)

#region_list = list(combined_result_dataframes.keys())


In [ ]:
#This block is to get all region names for placement into CCFv3 space
view_directory = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/MERFISH-C57BL6J-638850-CCF/20231215/views'
file = os.path.join( view_directory, 'cell_metadata_with_parcellation_annotation.csv')
cell_joined = pd.read_csv(file)
cell_joined.set_index('cell_label',inplace=True)


#Read CCFv3 annotation volume
data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
CCFv3_0, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3_0.nrrd')
# CCFv3_0, _ = nrrd.read("/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/annotation_ccfv3_l23split_barrelsplit_validated.nrrd"

save_nrrd = f'{root_folder}nrrd/'
# Specify the full or relative path to the log file



In [17]:
count = 0
for cluster, df in shuffled_combined_dataframes.items():
    print(f"Combined DataFrame for cluster '{cluster}':", flush=True)
    f_name = cluster.translate(str.maketrans({"/":  r"", "-":  r"_", " ":  r"_"}))
    created_nrrds, nrrd_files_without_extension = get_nrrd_files(save_nrrd)
    if f_name not in nrrd_files_without_extension:
        print(f"{f_name} is not created yet.", flush=True)

        #code
        result_CCFv3_0_copy = nrrd_from_df(df, description_of_all_indexes, CCFv3_0, region_map)

        print(f"Saving file: {cluster} as {f_name}" )
        print("\n", flush=True)
        nrrd.write(f"{save_nrrd}{f_name}.nrrd", result_CCFv3_0_copy)
        #iterate      
        count += 1
        print(count, flush=True)
        # if count == 1:
        #     break

    else:
        print(f"Exception occurred for cluster {cluster}", flush=True)
        # Handle the exception if needed
        print("\n", flush=True)
        continue

Combined DataFrame for cluster '0589 OB Dopa-Gaba_1':
0589_OB_Dopa_Gaba_1 is not created yet.
ACB 2 (region name and its subregions)
AIv1 1 (region name is a leaf region )
AIv23 1 (region name is a leaf region )
AOBgl 1 (region name is a leaf region )
AOBmi 1 (region name is a leaf region )
AON 2 (region name and its subregions)
CP 2 (region name and its subregions)
DP 2 (region name and its subregions)
IG 2 (region name and its subregions)
MOBgr 1 (region name is a leaf region )
MOBipl 1 (region name is a leaf region )
MOBmi 1 (region name is a leaf region )
MOBopl 1 (region name is a leaf region )
MOBunassigned 1 (region name is a leaf region )
MOp6b 1 (region name is a leaf region )
OLFunassigned 2 (region name and its subregions)
ORBl1 1 (region name is a leaf region )
ORBm1 1 (region name is a leaf region )
ORBvl1 1 (region name is a leaf region )
PIR 2 (region name and its subregions)
SSpll6a 1 (region name is a leaf region )
STRunassigned 2 (region name and its subregions)
TTd 1

KeyboardInterrupt: 

In [ ]:
nrrd_files_without_extension

## Test area

In [ ]:
type(shuffled_combined_dataframes.items())

In [ ]:
for cluster, df in shuffled_combined_dataframes.items():
    if cluster == '5252 Ependymal NN_1':
        print(f"Combined DataFrame for cluster '{cluster}':", flush=True)
        f_name = cluster.translate(str.maketrans({"/":  r"", "-":  r"_", " ":  r"_"}))
        #created_nrrds, nrrd_files_without_extension = get_nrrd_files(save_nrrd)
        all_ids_for_df = []
        df_comb = pd.DataFrame()
    
        for regionname in df.index.values:
            density = df.loc[regionname][0]        
        
        
        
        break
        

In [ ]:
df.loc[regionname, 'density_mm3']

In [ ]:
density

In [ ]:
import re

all_ids_for_df = []
df_comb = pd.DataFrame()

# for cluster, df in shuffled_combined_dataframes.items():
#     #if cluster == '3890 MB-MY Tph2 Glut-Sero_2':
#         print(f"Combined DataFrame for cluster '{cluster}':", flush=True)
#         f_name = cluster.translate(str.maketrans({"/":  r"", "-":  r"_", " ":  r"_"}))
for regionname in df.index.values:
        #if regionname == 'SSpbfd1':
        #if regionname == 'CS':
            #print(regionname)
            density = df.loc[regionname][0]
            #print(density)
            #annotation_id_info = substructure_info[substructure_info['cluster_as_filename'] == regionname]
            annotation_id_info = description_of_all_indexes[description_of_all_indexes['cluster_as_filename'] == regionname]
            Annotation2020ids = list(annotation_id_info['label_numbers'].values) 
            
            Annotation2020ids = [int(re.search(r'\d+$', s).group()) for s in annotation_id_info['parcellation_label'].values]
            #print(Annotation2020ids, annotation_id_info.shape)
            df_sub = pd.DataFrame({'density': density}, index=Annotation2020ids)
            df_comb = pd.concat([df_comb, df_sub])
    
            all_ids_for_df.append(Annotation2020ids)
            if annotation_id_info.shape[0] == 0:
                print(regionname, " region was not found and the region's density will be 0!!!", flush=True)
            elif annotation_id_info.shape[0] == 1:
                print(regionname, annotation_id_info.shape[0], "(region name is a leaf region )", flush=True)
            else:
                print(regionname, annotation_id_info.shape[0], "(region name and its subregions)", flush=True)
all_ids_for_df = [value for sublist in all_ids_for_df for value in sublist]
all_ids_for_df.append(0)

outside = 0
outsideid = [0]
df_sub = pd.DataFrame({'density': outside}, index=outsideid)
df_comb = pd.concat([df_comb, df_sub])

In [ ]:
count = 0
for cluster, df in shuffled_combined_dataframes.items():
    if cluster == '0629 Vip Gaba_2':
        
        print(f"Combined DataFrame for cluster '{cluster}':", flush=True)
        f_name = cluster.translate(str.maketrans({"/":  r"", "-":  r"_", " ":  r"_"}))
        #created_nrrds, nrrd_files_without_extension = get_nrrd_files(save_nrrd)
        
        all_ids_for_df = []
        df_comb = pd.DataFrame()
        
        for regionname in df.index.values:
            density = df.loc[regionname][0]
            annotation_id_info = description_of_all_indexes[description_of_all_indexes['parcellation_term_acronym'] == regionname]
            Annotation2020ids = [int(re.search(r'\d+$', s).group()) for s in annotation_id_info['parcellation_label'].values]
            df_sub = pd.DataFrame({'density': density}, index=Annotation2020ids)
            df_comb = pd.concat([df_comb, df_sub])
        
            all_ids_for_df.append(Annotation2020ids)
        
            print(regionname, annotation_id_info.shape[0], "(region name and its subregions)", flush=True)
        
        all_ids_for_df = [value for sublist in all_ids_for_df for value in sublist]
        all_ids_for_df.append(0)
        
        outside = 0
        outsideid = [0]
        df_sub = pd.DataFrame({'density': outside}, index=outsideid)
        df_comb = pd.concat([df_comb, df_sub])
        
        
        
        
        
        
        
        
        
        # print(f"{f_name} is not created yet.", flush=True)
        
        # #code
        # result_CCFv3_0_copy = nrrd_from_df(df, description_of_all_indexes, CCFv3_0)
        
        # print(f"Saving file: {cluster} as {f_name}" )
        # print("\n", flush=True)
        #nrrd.write(f"{save_nrrd}{f_name}.nrrd", result_CCFv3_0_copy)
        #iterate      
        count += 1
        print(count, flush=True)
        if count == 1:
            break

    else:
        None


In [ ]:
cluster, f_name

In [ ]:
substructure_info['label_numbers'] = substructure_info['parcellation_label'].str.extract(r'AllenCCF-Annotation-2020-(\d+)')
substructure_info['label_numbers'] = substructure_info['label_numbers'].astype(int)

In [ ]:
substructure_info['cluster_as_filename'] = substructure_info['parcellation_term_acronym'].str.translate(str.maketrans({"/":  r"", "-":  r"", " ":  r"", ",": r""}))

In [ ]:
parcellation_annotation['label_numbers'] = parcellation_annotation['parcellation_label'].str.extract(r'AllenCCF-Annotation-2020-(\d+)')
parcellation_annotation['label_numbers'] = parcellation_annotation['label_numbers'].astype(int)
parcellation_annotation['cluster_as_filename'] = parcellation_annotation['parcellation_term_acronym'].str.translate(str.maketrans({"/":  r"", "-":  r"", " ":  r"", ",": r""}))

In [ ]:
parcellation_annotation

In [ ]:
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation.to_csv(file)

In [ ]:
cells_in_region[['z_section', 'template_nr', 'best_position']]

In [ ]:
CCFv3_0_copy = np.copy(CCFv3_0).astype(float)

In [ ]:
# Create a boolean mask where True indicates the elements that match the target value
mask = (CCFv3_0_copy == region_id)

# Set all elements to 0 except the ones that match the target value
CCFv3_0_copy[~mask] = 0

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(CCFv3_0_copy[z_index]);
plt.title(f"z_index is {z_index}");

In [ ]:
np.unique(np.where(CCFv3_0_copy == 393)[0])

In [ ]:
## Continuing here

In [ ]:
CCFv3_0_copy = np.copy(CCFv3_0).astype(float)

# Convert angle to radians
intersection_angle_rad = np.radians(intersection_angle_deg)

# Define dimensions of the volume
volume_shape = CCFv3_0_copy.shape
array2D_width, array2D_height = volume_shape[2], volume_shape[1]

# Create meshgrid for x and y coordinates
x_coordinates, y_coordinates = np.meshgrid(np.arange(array2D_width), np.arange(array2D_height))

# Calculate y coordinate of the line at each x coordinate based on intersection angle
y_line = np.tan(intersection_angle_rad) * np.arange(array2D_width)
y_line_int_clipped = np.clip(np.round(y_line).astype(int), 0, array2D_height - 1)

# Create boolean mask to identify voxels in intersection region
mask = (y_coordinates <= y_line_int_clipped)

# Multiply voxels in intersection region by 2
CCFv3_0_copy[z_index, :, :][~mask] = 0
CCFv3_0_copy[z_index, :, :][mask] *= 2

# Count number of voxels affected by the plane
intersection_vol_in_voxels = np.count_nonzero(CCFv3_0_copy[z_index, :, :] == region_id*2)


In [ ]:
x_coordinates.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_coordinates);

In [ ]:
plt.imshow(CCFv3_0_copy[402], cmap='coolwarm');